# The Battle Of Neighborhoods - Week1

## 1.1 Introduction

The purpose of this report is to find the best location to open a sushi restaurant in Manhattan, New York. This report is aimed at people who want to open a sushi restaurant in Manhattan.

## 1.2 Data

To solve this problem, we will use the following data
* Neighborhoods in New York
* Latitude and Longitude of Neighborhoods in New York
* The sushi restaurant opened in Manhattan

To solve the problem, I will:
* Scrape https://cocl.us for all neighborhoods in New York
* Obtain latitude and longitude of those neighborhoods using the Geocoder package
* Use the Foursquare API to retrieve Japanese restaurant venues in New York


# The Battle Of Neighborhoods - Week2

In [14]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans
import wget

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: ...working... done

# All requested packages already installed.



In [5]:
newyork_data=pd.read_csv('https://cocl.us/new_york_dataset')

In [6]:
newyork_data.head()

,"{""type"":""FeatureCollection""",totalFeatures:306,"features:[{""type"":""Feature""","id:""nyu_2451_34572.1""","geometry:{""type"":""Point""",coordinates:[-73.84720052054902,40.89470517661]},"geometry_name:""geom""","properties:{""name"":""Wakefield""",stacked:1,"annoline1:""Wakefield""",annoline2:null,annoline3:null,annoangle:0E-11,"borough:""Bronx""",bbox:[-73.84720052054902,40.89470517661,-73.84720052054902,40.89470517661]}},"{""type"":""Feature""","id:""nyu_2451_34572.2""","geometry:{""type"":""Point"".1",coordinates:[-73.82993910812398,40.87429419303012]},"geometry_name:""geom"".1","properties:{""name"":""Co-op City""",stacked:2,"annoline1:""Co-op""","annoline2:""City""",annoline3:null.1,annoangle:0E-11.1,"borough:""Bronx"".1",bbox:[-73.82993910812398,40.87429419303012,-73.82993910812398,40.87429419303012]}},"{""type"":""Feature"".1","id:""nyu_2451_34572.3""","geometry:{""type"":""Point"".2",coordinates:[-73.82780644716412,40.887555677350775]},"geometry_name:""geom"".2","properties:{""name"":""Eastchester""",stacked:1.1,"annoline1:""Eastchester""",annoline2:null.1,annoline3:null.2,annoangle:0E-11.2,"borough:""Bronx"".2",bbox:[-73.82780644716412,40.887555677350775,-73.82780644716412,40.887555677350775]}},"{""type"":""Feature"".2","id:""nyu_2451_34572.4""","geometry:{""type"":""Point"".3",coordinates:[-73.90564259591682,40.89543742690383]},"geometry_name:""geom"".3","properties:{""name"":""Fieldston""",stacked:1.2,"annoline1:""Fieldston""",annoline2:null.2,annoline3:null.3,annoangle:0E-11.3,"borough:""Bronx"".3",bbox:[-73.90564259591682,40.89543742690383,-73.90564259591682,40.89543742690383]}},"{""type"":""Feature"".3","id:""nyu_2451_34572.5""","geometry:{""type"":""Point"".4",coordinates:[-73.9125854610857,40.890834493891305]},"geometry_name:""geom"".4","properties:{""name"":""Riverdale""",stacked:1.3,"annoline1:""Riverdale""",annoline2:null.3,annoline3:null.4,annoangle:0E-11.4,"borough:""Bronx"".4",bbox:[-73.9125854610857,40.890834493891305,-73.9125854610857,40.890834493891305]}},"{""type"":""Feature"".4","id:""nyu_2451_34572.6""","geometry:{""type"":""Point"".5",coordinates:[-73.90281798724604,40.88168737120521]},"geometry_name:""geom"".5","properties:{""name"":""Kingsbridge""",stacked:1.4,"annoline1:""Kingsbridge""",annoline2:null.4,annoline3:null.5,annoangle:0E-11.5,"borough:""Bronx"".5",bbox:[-73.90281798724604,40.88168737120521,-73.90281798724604,40.88168737120521]}},"{""type"":""Feature"".5","id:""nyu_2451_34572.7""","geometry:{""type"":""Point"".6",coordinates:[-73.91065965862981,40.87655077879964]},"geometry_name:""geom"".6","properties:{""name"":""Marble Hill""",stacked:2.1,"annoline1:""Marble""","annoline2:""Hill""",annoline3:null.6,annoangle:0E-11.6,"borough:""Manhattan""",bbox:[-73.91065965862981,40.87655077879964,-73.91065965862981,40.87655077879964]}},"{""type"":""Feature"".6","id:""nyu_2451_34572.8""","geometry:{""type"":""Point"".7",coordinates:[-73.86731496814176,40.89827261213805]},"geometry_name:""geom"".7","properties:{""name"":""Woodlawn""",stacked:1.5,"annoline1:""Woodlawn""",annoline2:null.5,annoline3:null.7,annoangle:0E-11.7,"borough:""Bronx"".6",bbox:[-73.86731496814176,40.89827261213805,-73.86731496814176,40.89827261213805]}},"{""type"":""Feature"".7","id:""nyu_2451_34572.9""","geometry:{""type"":""Point"".8",coordinates:[-73.8793907395681,40.87722415599446]},"geometry_name:""geom"".8","properties:{""name"":""Norwood""",stacked:1.6,"annoline1:""Norwood""",annoline2:null.6,annoline3:null.8,annoangle:0E-11.8,"borough:""Bronx"".7",bbox:[-73.8793907395681,40.87722415599446,-73.8793907395681,40.87722415599446]}},"{""type"":""Feature"".8","id:""nyu_2451_34572.10""","geometry:{""type"":""Point"".9",coordinates:[-73.85744642974207,40.88103887819211]},"geometry_name:""geom"".9","properties:{""name"":""Williamsbridge""",stacked:1.7,"annoline1:""Williamsbridge""",annoline2:null.7,annoline3:null.9,annoangle:0E-11.9,"borough:""Bronx"".8",bbox:[-73.85744642974207,40.88103887819211,-73.85744642974207,40.88103

In [8]:
pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9684 sha256=2a0cf34cfbb544102a5d6e2242722e7c367a952cfe3efda7c76e1fa49b1247bc
  Stored in directory: c:\users\watermelon\appdata\local\pip\cache\wheels\a1\b6\7c\0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\WATERMELON\AppData\Local\Continuum\anaconda3\python.exe -m pip install --upgrade pip' command.


In [23]:
resp = urllib.request.urlopen("https://cocl.us/new_york_dataset")
newyork_data = json.loads(resp.read())

In [24]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [25]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [26]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [27]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [28]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [29]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [30]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'ZMHWBS0SR12Z3YDYVHJVTZPRK3U1ZP3I2TYQAJ5CU3JUHMB5'
CLIENT_SECRET = 'H3TT0XT3P5TIAFCV1Y2UUVLF42N44DICNKLUELK34H2TKLFR'
VERSION = '20181020'

In [31]:
#https://developer.foursquare.com/docs/resources/categories
#Sushi = 4bf58dd8d48988d1d2941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_sushi = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1d2941735')
newyork_venues_sushi.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,Planet Tokyo,40.886233,-73.909479,Sushi Restaurant
1,Chinatown,40.715618,-73.994279,Nakaji,40.715912,-73.996597,Sushi Restaurant
2,Chinatown,40.715618,-73.994279,Shinsen,40.715608,-73.996611,Japanese Restaurant
3,Chinatown,40.715618,-73.994279,Bondi Bar,40.721247,-73.996264,Sushi Restaurant
4,Chinatown,40.715618,-73.994279,Whole Foods Sushi Bar,40.724000,-73.992277,Sushi Restaurant


In [32]:
newyork_venues_sushi.shape

(1100, 7)

In [33]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [34]:
map_newyork_sushi = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_sushi, 'red', map_newyork_sushi)

map_newyork_sushi

In [35]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [36]:
manhattan_grouped = newyork_venues_sushi.groupby('Neighborhood').count()
manhattan_grouped
#print('There are {} uniques categories.'.format(len(newyork_venues_sushi['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,22,22,22,22,22,22
Carnegie Hill,24,24,24,24,24,24
Central Harlem,4,4,4,4,4,4
Chelsea,44,44,44,44,44,44
Chinatown,24,24,24,24,24,24
Civic Center,32,32,32,32,32,32
Clinton,38,38,38,38,38,38
East Harlem,4,4,4,4,4,4
East Village,49,49,49,49,49,49


### Analyze Each Neighborhood

In [37]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_sushi[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_sushi['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Asian Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant,Noodle House,Ramen Restaurant,Restaurant,Sake Bar,Sandwich Place,Seafood Restaurant,Smoothie Shop,Steakhouse,Sushi Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,Chinatown,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [38]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head()

,Neighborhood,Asian Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant,Noodle House,Ramen Restaurant,Restaurant,Sake Bar,Sandwich Place,Seafood Restaurant,Smoothie Shop,Steakhouse,Sushi Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant
0,Battery Park City,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.090909,0.045455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.818182,0.045455,0.000000
1,Carnegie Hill,0.041667,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.041667,0.125000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.750000,0.000000,0.041667
2,Central Harlem,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.000000
3,Chelsea,0.045455,0.0,0.0,0.0,0.0,0.022727,0.0,0.0,0.000000,0.113636,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.795455,0.000000,0.022727
4,Chinatown,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.125000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.875000,0.000000,0.000000


In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Sushi Restaurant,Japanese Restaurant,Noodle House,Theme Restaurant,Indian Chinese Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market
1,Carnegie Hill,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Indian Chinese Restaurant,Asian Restaurant,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
2,Central Harlem,Sushi Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
3,Chelsea,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Fish Market,Vegetarian / Vegan Restaurant,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
4,Chinatown,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant


In [41]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 0, 2, 2, 0, 1, 2, 2])

In [42]:

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Sushi Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,2,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,1,Sushi Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,1,Sushi Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Sushi Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant


In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [44]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Smoothie Shop,Hawaiian Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega
9,Yorkville,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Indian Chinese Restaurant,Asian Restaurant,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
10,Lenox Hill,Sushi Restaurant,Asian Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Smoothie Shop,Hawaiian Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega
13,Lincoln Square,Sushi Restaurant,Japanese Restaurant,Smoothie Shop,Chinese Restaurant,Grocery Store,Vegetarian / Vegan Restaurant,Indian Chinese Restaurant,Bakery,Cocktail Bar,Deli / Bodega
14,Clinton,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Chinese Restaurant,Seafood Restaurant,Cocktail Bar,Indian Chinese Restaurant,Bakery,Deli / Bodega,Fish Market
15,Midtown,Sushi Restaurant,Asian Restaurant,Seafood Restaurant,Japanese Restaurant,Restaurant,Bakery,Ramen Restaurant,Vegetarian / Vegan Restaurant,Sandwich Place,Sake Bar
16,Murray Hill,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Restaurant,Bakery,Ramen Restaurant,Vegetarian / Vegan Restaurant,Sake Bar,Sandwich Place,Seafood Restaurant
17,Chelsea,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Fish Market,Vegetarian / Vegan Restaurant,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
28,Battery Park City,Sushi Restaurant,Japanese Restaurant,Noodle House,Theme Restaurant,Indian Chinese Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market
30,Carnegie Hill,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Indian Chinese Restaurant,Asian Restaurant,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant


In [45]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sushi Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
2,Washington Heights,Sushi Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
3,Inwood,Sushi Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
4,Hamilton Heights,Sushi Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
5,Manhattanville,Sushi Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
6,Central Harlem,Sushi Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
7,East Harlem,Sushi Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
18,Greenwich Village,Sushi Restaurant,Sake Bar,Japanese Restaurant,Vegetarian / Vegan Restaurant,Indian Chinese Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market


In [46]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
11,Roosevelt Island,Sushi Restaurant,Asian Restaurant,Japanese Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
19,East Village,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
21,Tribeca,Sushi Restaurant,Noodle House,Theme Restaurant,Indian Chinese Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store
22,Little Italy,Sushi Restaurant,Japanese Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
23,Soho,Sushi Restaurant,Japanese Restaurant,Noodle House,Theme Restaurant,Indian Chinese Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market
24,West Village,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Sake Bar,Fish Market,Asian Restaurant,Sandwich Place,Seafood Restaurant,Restaurant,Ramen Restaurant
27,Gramercy,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
29,Financial District,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
31,Noho,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant


In [47]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Lower East Side,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
25,Manhattan Valley,Sushi Restaurant,Japanese Restaurant,Hawaiian Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store
26,Morningside Heights,Sushi Restaurant,Japanese Restaurant,Hawaiian Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store


In [48]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Upper West Side,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Grocery Store,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Hawaiian Restaurant
